# W266 Final Project

Authors: Satheesh Joseph, Catherine Mou, Yi Zhang

## Downloading and loading the data

We acquired the dataset from the researchers in the form of Sqlite `.db` files.

In [1]:
import os, sys, re, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
from scipy import stats
import pandas as pd
import sqlite3

import nltk

In [2]:
# Download the files if they're not here
if 'data' not in os.listdir('.') or not os.listdir('data'):
    os.system('wget https://storage.googleapis.com/mids-w266-final-project-data/yelpHotelData.db -P data/')
    os.system('wget https://storage.googleapis.com/mids-w266-final-project-data/yelpResData.db -P data/')
    print('Data downloaded successfully!')
else:
    print('Already downloaded data')

In [3]:
# Reading from the hotels database
hotels_db = sqlite3.connect("data/yelpHotelData.db")
hotels = pd.read_sql_query("SELECT * FROM hotel", hotels_db)
hotel_reviews = pd.read_sql_query("SELECT * FROM review", hotels_db)
hotel_reviewers = pd.read_sql_query("SELECT * FROM reviewer", hotels_db)

print(f'The data set contains {len(hotels)} hotels, {len(hotel_reviews)} reviews, and {len(hotel_reviewers)} reviewers')

The data set contains 283086 hotels, 688329 reviews, and 5123 reviewers


In [4]:
# Reading from the restaurant database
restaurant_db = sqlite3.connect("data/yelpResData.db")
restaurant_db.text_factory = bytes 
restaurants = pd.read_sql_query("SELECT * FROM restaurant", restaurant_db)
restaurant_reviews = pd.read_sql_query("SELECT * FROM review", restaurant_db)
restaurant_reviewers = pd.read_sql_query("SELECT * FROM reviewer", restaurant_db)

# TODO: UTF-8 decode the contents
print(f'The data set contains {len(restaurants)} restaurants, {len(restaurant_reviews)} reviews, and {len(restaurant_reviewers)} reviewers')

The data set contains 242652 restaurants, 788471 reviews, and 16941 reviewers


# Exploratory Data Analysis

## ToDo: Performan EDA

In [5]:
hotel_reviews.head()

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,hotelID
0,6/8/2011,MyNjnxzZVTPq,IFTr6_6NI4CgCVavIL9k5g,Let me begin by saying that there are two kind...,5,18,11,28,N,tQfLGoolUMu2J0igcWcoZg
1,8/30/2011,BdD7fsPqHQL73hwENEDT-Q,c_-hF15XgNhlyy_TqzmdaA,The only place inside the Loop that you can st...,3,0,3,4,N,tQfLGoolUMu2J0igcWcoZg
2,6/26/2009,BfhqiyfC,CiwZ6S5ZizAFL5gypf8tLA,I have walked by the Tokyo Hotel countless tim...,5,12,14,23,N,tQfLGoolUMu2J0igcWcoZg
3,9/16/2010,Ol,nf3q2h-kSQoZK2jBY92FOg,"If you are considering staying here, watch thi...",1,8,2,6,N,tQfLGoolUMu2J0igcWcoZg
4,2/5/2010,i4HIAcNTjabdpG1K4F5Q2g,Sb3DJGdZ4Rq__CqxPbae-g,"This place is disgusting, absolutely horrible,...",3,11,4,9,N,tQfLGoolUMu2J0igcWcoZg


In [6]:
restaurant_reviews.head()

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,restaurantID
0,b'9/22/2012',b'GtwU21YOQn-wf4vWRUIx6w',b'bNYesZ944s6IJVowOnB0iA',"b'Unlike Next, which we\'d eaten at the previo...",5,0,0,0,b'N',b'pbEiXam9YJL3neCYHGwLUA'
1,b'9/22/2012',b'0LpVTc3',b'TRKxLC3y-ZvP45e5iilMtw',"b""Probably one of the best meals I've had ever...",5,0,0,0,b'N',b'pbEiXam9YJL3neCYHGwLUA'
2,b'9/19/2012',b'tljtLzf68Fkwf',b'0EMm8umAqXZzyhxNpL4M9g',"b""Service was impeccable. Experience and prese...",3,2,0,0,b'N',b'pbEiXam9YJL3neCYHGwLUA'
3,b'9/6/2012',b'iSN',b'DlwexC7z88ymAzu45skODw',"b""The problem with places like this, given the...",3,8,0,3,b'N',b'pbEiXam9YJL3neCYHGwLUA'
4,b'9/9/2012',b'Jmwrh7',b'kW2dk1CWihmh3g7k9N2G8A',"b""I have no idea how to write my review - dini...",5,1,2,0,b'N',b'pbEiXam9YJL3neCYHGwLUA'
